In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

import dateutil.easter as easter
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator, FormatStrFormatter, PercentFormatter
from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor

# EDA
- https://www.kaggle.com/ambrosm/tpsjan22-01-eda-which-makes-sense/notebookhttps://www.kaggle.com/ambrosm/tpsjan22-01-eda-which-makes-sense/notebook 참고하세요
- 시즌별, 요일별 배리에이션
- 이스터의 효과
- 주말 피크를 자세히 보기
- 연간 성장(연말에 더 높음)
- 왜 SMAPE로 점수를 매기나

In [ ]:
train_df = pd.read_csv('../input/tabular-playground-series-jan-2022/train.csv')
test_df = pd.read_csv('../input/tabular-playground-series-jan-2022/test.csv')

for df in [train_df, test_df]:
    df['date'] = pd.to_datetime(df.date)
    df.set_index('date', inplace=True, drop=False)
    
train_df

# 데이터의 완결성
트레인 데이터엔 세개의 나라, 두개의 가게, 세개의 상품이있다. 
18개의 조합의 1461일의 판매 데이터가 있다.
1461일은 2015,2016, 2017, 2018 년에 포함된다.
결측값은 없다.

In [ ]:
print(train_df.groupby(['country', 'store', 'product']).date.count())

print("First day:", train_df.date.min(), "  \n Last day:", train_df.date.max())
print("Number of days in four years:", 365 * 4 + 1) # 윤년이 포함됨
print(18 * 1461, train_df.shape, train_df.date.isna().sum())

모든 제품이 모든 가게에서 모든 날에 팔렸다. 그러므로 판매량은 항상 양의 값을 가질 것이다.

In [ ]:
train_df.groupby(['country', 'store', 'product']).num_sold.agg(['min', 'max', 'mean'])

학습 데이터는 2015년부터 2018년까지를 다룬다면, 테스트 데이터는 2019년의 값은 구하는 것을 요구할 것이다.

In [ ]:
test_df.date.min(), test_df.date.max()

# KaggleRama가 직관적으로 봐도 더 많이 팔린다.
모든 국가에서 상품에서 캐글라마가 캐글마트보다 더많이 판다.

둘 사이의 관계가 예측값을 구하는 힌트가 될 수도 있다.

In [ ]:
kk = train_df.groupby(['country', 'store', 'product']).num_sold.mean().unstack(level='store')
kk['KaggleRama:KaggleMart'] = kk.KaggleRama / kk.KaggleMart
kk

# Product

- 데이터를 국가별로, 년도별로 상품별로 그루핑했을 때 Sticker:Hat:Mug 의 비율이 거의 비슷하다. 
- 이 비율은 년도나 국가나 가게에게 큰 영향을 받지 않는 경향을 보인다.
- 하지만 월별, 주별로 그루핑했을 때는 다른 결과를 보이기 때문에 시즌별로 다른 경향을 보이는 것 같다.

In [ ]:
# group by week
kk = train_df.groupby(['country', 'store', 'product', train_df.date.dt.week]).num_sold.mean().unstack(level='product')
kk['Mug/Sticker'] = kk['Kaggle Mug'] / kk['Kaggle Sticker']
kk['Hat/Sticker'] = kk['Kaggle Hat'] / kk['Kaggle Sticker']
kk.head(30)

In [ ]:
# group by month
kk = train_df.groupby(['country', 'store', 'product', train_df.date.dt.month]).num_sold.mean().unstack(level='product')
kk['Mug/Sticker'] = kk['Kaggle Mug'] / kk['Kaggle Sticker']
kk['Hat/Sticker'] = kk['Kaggle Hat'] / kk['Kaggle Sticker']
kk

In [ ]:
# group by year
kk = train_df.groupby(['country', 'store', 'product', train_df.date.dt.year]).num_sold.mean().unstack(level='product')
kk['Mug/Sticker'] = kk['Kaggle Mug'] / kk['Kaggle Sticker']
kk['Hat/Sticker'] = kk['Kaggle Hat'] / kk['Kaggle Sticker']
kk

# Histograms and SMAPE
- 18개의 country-store-product 조합은 모두 다르지만 대체로 히스토그램 그래프가 편향된 모습을 보여준다.
- 모든 상품은 평균을 한참 뛰어넘게 팔리는 날들이 있다.
- 이 아웃라이어들이 보통 날들에 비해 예측을 부정확하게 할 수도 있다.

### Insight
- 이런 이유들 때문에 SMAPE Evaluation 방법을 쓰는 것 같다.
- 그래서 적절한 손실 함수를 고르는 것이 중요하다.
- log를 예측하는 것이 num_sold를 예측하는 것보다 더 나을 수 있다.
- 

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.hist(df.num_sold, bins=50, color='pink')
    ax.set_title(combi)
    
plt.suptitle("Histogram of num_sold", y=1.03)
plt.tight_layout(h_pad=3.0)
plt.show()

# Time series
## Daily sales and the year-end peak
- 일별로 조합의 일별 num_sold를 보면 각 년도의 말기에 수치가 높아지는 것을 확인할 수 있다.
- 상품별로 그래프의 특징이 있는데 
    - Kaggle Hat은 겨울철에 판매량이 증가하는 경향을 보이고,
    - Mug는 Norway, Finland, Sweden 순으로 판매량이 많은 것을 보면 국가별 차이가 있는 것으로 보이고
    - Sticker는 대체로 그래프가 평탄하지만 Norway의 2분기에 매출량이 증가하는 것이 보인다.
    
### Insight
- model에 계절별 특징을 반영할 수 있어야 한다.
- 년 단위의 그래프 뿐만 아니라 짧은 spike 들도 반영해야 된다.

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.plot(df.num_sold)
    ax.set_title(combi)
    
plt.tight_layout(h_pad=0.3)
plt.suptitle('Daily sales for 2015-2018', y=1.03)
plt.show()

- 4년간 매출량의 12월 평균을 내어 연말의 상황을 살펴보자
- 노란색 부분을 보면 크리스마스 이후에 매출량이 증가하는 것을 볼 수 있다.

In [ ]:
plt.figure(figsize=(18,12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    ax.bar(range(1, 32),
          df.num_sold[df.date.dt.month==12].groupby(df.date.dt.day).mean(),
          color=['b'] * 25 + ['y'] * 6)
    ax.set_title(combi)
    ax.set_xticks(ticks=range(5, 31, 5))

plt.tight_layout(h_pad=0.3)
plt.suptitle('Daily sales for December', y=1.03)
plt.show()
        

## Monthly sales and seasonal variation
- 월별로 매출량을 분석했을 때 계절별 변화와 성장 추세를 확인할 수 있다.
- Sticker가 hat에 비해 증가세가 뚜렷해 보인다.

### Insight
- 증가 추세를 통해 다음년도(2019)의 값을 예측할 수도 있다.

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    resampled = df.resample('MS').num_sold.sum()
    ax.bar(range(len(resampled)), resampled)
    ax.set_title(combi)
    ax.set_ylim(resampled.min(), resampled.max())
    ax.set_xticks(range(0, 48, 12), [f"Jan {y}" for y in range(2015, 2019)])
    
plt.suptitle('Monthly sales for 2015-2018', y = 1.03)
plt.tight_layout(h_pad=3.0)
plt.show()

- Hat은 대체적으로 4,5 월에 최대치를 찍고 9월달에 최소치를 찍는다.
- Mug는 12월달에 최대치를 찍고, 6,7월달에 최소치를 찍는다.
- Sticker는 12,1월에 최대치를 찍고, 2월달에 최소치를 찍는다.(5월달에도 스파이크가 있다.)

In [ ]:
plt.figure(figsize=(18,12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    resampled = df.resample('MS').sum()
    resampled = resampled.groupby(resampled.index.month).mean()
    ax.bar(range(1, 13), resampled.num_sold)
    ax.set_xticks(ticks=range(1,13), labels='JFMAMJJASOND')
    ax.set_title(combi)
    ax.set_ylim(resampled.num_sold.min(), resampled.num_sold.max())

plt.suptitle('Monthly sales for 2015-2018', y=1.03)
plt.tight_layout(h_pad=3.0)
plt.show()


## Growth
- 년도별로 매출량을 합하면 성장 추세를 확인할 수 있다.(계절별 변화 무시 가능)
    - Norway는 2016년에 살짝 마이너스 성장했다.
    - Sweden은 2018년 성장이 거의 없다.
    - 대체적으로 매년 성장한다.
    
### insight
- 국가별로 성장률을 모델링해야함
- 성장은 지수성장이나 선형 성장이 아니다.
- 연말에 매출이 몰리는 것을 감안해 다른 성장률도 모델링해야함

In [ ]:
plt.figure(figsize=(18, 12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    resampled = df.resample('AS').sum()
    ax.bar(range(2015, 2019), resampled.num_sold, color='brown')
    ax.set_title(combi)
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
    ax.set_ylim(0, resampled.num_sold.max())
plt.suptitle('Growth of yearly sales for 2015-2018', y=1.03)
plt.tight_layout(h_pad=3.0)
plt.show()

In [ ]:
plt.figure(figsize=(18, 90))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(18, 1, i+1, ymargin=0.5)
    resampled = df[(df.date.dt.month<12) | (df.date.dt.day<25)].resample('AS').num_sold.sum()
    resampled /= resampled.iloc[0]
    resampled_end_of_year = df[(df.date.dt.month==12) | (df.date.dt.day>=25)].resample('AS').num_sold.sum()
    resampled_end_of_year /= resampled_end_of_year.iloc[0]
    ax.bar(range(2015, 2019), resampled, color='brown')
    ax.bar(range(2015, 2019), resampled_end_of_year, color='orange', width=0.4)
    
    X = np.arange(2015, 2019).reshape(-1, 1)
    lr = TransformedTargetRegressor(LinearRegression(), func=np.log, inverse_func=np.exp)
    lr.fit(X, resampled)
    ax.plot(range(2015, 2019), lr.predict(X), color='brown', label=f"whole year: {lr.predict([[2018]]).squeeze() - 1:.1%}")
    lr.fit(X, resampled_end_of_year)
    ax.plot(range(2015, 2019), lr.predict(X), color='orange', label = f"end of year: {lr.predict([[2018]]).squeeze() - 1:.1%}")
    
    ax.legend()
    ax.set_title(f"Yearly sales for {combi}")
    ax.xaxis.set_major_locator(MaxNLocator(integer=True))
plt.tight_layout(h_pad=3.0)
plt.show()

## Weekdays
- 모든 상품에서 주말에 매출량이 제일 많다
- 주중에서도 금요일이 가장 매출량이 좋다.

### insight
- 우리의 모델은 적어도 월-목, 금, 토-일 을 구분이 되어야한다.

In [ ]:
plt.figure(figsize=(12,12))
for i, (combi, df) in enumerate(train_df.groupby(['country', 'store', 'product'])):
    ax = plt.subplot(6, 3, i+1, ymargin=0.5)
    resampled = df.groupby(df.index.dayofweek).mean()
    ax.bar(range(7), resampled.num_sold,
           color = ['b'] * 4 + ['g'] + ['orange'] * 2)
    ax.set_title(combi)
    ax.set_xticks(ticks=range(7), labels=('M', 'T', 'W', 'T', 'F', 'S', 'S'))
    ax.set_ylim(0, resampled.num_sold.max())
plt.suptitle('Sales per day of the week', y= 1.03)
plt.tight_layout(h_pad=3.0)
plt.show()

## Easter
- 부활절 이후의 매출량이 평소보다 높다.
- 

### insight
- 모델은 부활절의 날짜를 알아야하고 부활절 다음에 수요가 많음을 인식해야함

In [ ]:
plt.figure(figsize=(12,12))
for i, (year, df) in enumerate(train_df.groupby(train_df.date.dt.year)):
    df = df.reset_index(drop=True)
    ax = plt.subplot(4, 1, i+1, ymargin=0.5)
    april = df.num_sold[(df.date.dt.month==4)].groupby(df.date.dt.day).mean()
    date_range = pd.date_range(start=f'{year}-04-01', end=f'{year}-04-30', freq='D')
    easter_date = easter.easter(year)
    color = ['r' if d == easter_date else 'lightblue' if (d.date() - easter_date).days in range(6) else 'b' if d.dayofweek < 5 else 'orange' for d in date_range]
    ax.bar(range(1,31),
          april,
          color=color)
    ax.set_title(str(year))
    ax.set_xticks(ticks=range(5, 31, 5))
plt.tight_layout(h_pad=3.0)
plt.suptitle('Daily sales for April', y=1.03)
plt.show()